**<p style = "text-align: center">TRƯỜNG ĐẠI HỌC KHOA HỌC TỰ NHIÊN - ĐẠI HỌC QUỐC GIA TPHCM</p>**

# **<p style = "text-align: center">NHẬP MÔN KHOA HỌC DỮ LIỆU</p>**
## **<p style = "text-align: center">Đồ án Cuối kì</p>**
## **<p style = "text-align: center">TRẢ LỜI CÂU HỎI</p>**

### <p style = "text-align: center"> <b>Đề tài:</b> Phân tích, khám phá dữ liệu về sách trên web thương mại điện tử (tiki)</p>
**Sinh viên thực hiện**

| Mã số sinh viên | Họ và tên |
| --- | --- |
| 20120040 | Nguyễn Quang Gia Bảo |
| 20120136 | Huỳnh Tuấn Nam |
| 20120158 | Trần Hoàng Anh Phi |

---

### Cài đặt thư viện khai báo

In [31]:
# !pip install fuzzywuzzy
# !pip install python-Levenshtein

### Khai báo thư viện sử dụng

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [33]:
df = pd.read_csv("./features/processed.csv")

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2140\316702570.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./features/processed.csv")


---

## **Trả lời câu hỏi**

### **Câu 1: Tại sao có những quyển sách trùng tên ?**
- Ở mỗi thể loại sách có rất nhiều sách khác nhau. Có nhiều sách cùng tên với nhau hay không? Tại sao lại như vậy?
<br>

**Trả lời câu hỏi thành công** <br>
Nếu trả lời được câu hỏi bạn sẽ hiểu tại sao trong bộ dữ liệu lại có nhiều sách có cùng tên (Nếu có)<br>

<font color = red>Khó khăn </font>: 
+ Tên của những quyển sách lại được đặt khác nhau. Ảnh hưởng đến quá trình gom nhóm. Cần xử lí vấn đề khó này cho một bộ dữ liệu gần 40 ngàn dữ liệu.<br>
+ Không có các yếu tố để phân biệt tên của sách.<br>

<font color = red>Hướng giải quyết </font>: 
+ Thay vì xử lí một lúc 40 ngàn dữ liệu, ta sẽ sử dụng cột `categories` để phân loại thành từng loại sách, rồi tiếp tục phân loại theo tên.
+ Ta sử dụng `Ratio Matching` - Độ tương thích của 2 string, để phán đoán series của cuốn truyện.


Dùng thử viện `fuzzywuzzy` để tìm mức độ tương thích, lọc ra các nhóm có mức độ tương thích cao.

In [34]:
from fuzzywuzzy import fuzz, process

##### Viết các hàm xử lí 

 - Hàm lấy các dòng thuộc `categories` mà ta yêu cầu.

In [35]:
def get_df_by_categories(categories):
    categories_book_df = df[df.categories == categories]
    return categories_book_df

- Sử dụng Data Frame mà ta vừa lấy được từ `categories` mà ta chọn, tiến hành phân theo mức độ tương thích của các tên với nhau.


Ví dụ sử dụng fuzzywuzzy để đánh giá mức tương thích:

In [36]:
# Ta có 2 string:
a = "Xin Cảm ơn"
b = "Xin Chào"

fuzz.ratio(a,b) # Độ giống nhau là 48%
# Độ tương thích này được xét theo các yếu tố như:
# Số lượng từ ở trong hai câu
# Vị trí từ
# Kiểu viết của chữ (hoa hoặc thường)
# .....

56

In [37]:
a = "Xin Cảm ơn"
b = "Xin Chào"

fuzz.partial_ratio(a ,b) # có bao nhiêu % của a trong b hoặc b trong a, dù đổi vị trí các parameter cũng không làm thay đổi giá trị %.
# Ở câu này do vị trí của từ Xin C

62

- Viết hàm phân nhóm theo series của các sách trong 1 categories.<br>
<br>

<font color = red>Cách làm </font>:<br>
- Ta sẽ kiểm tra có 1 `list` để chứa các series và sẽ dùng `list` này để đi so sánh với tên của các cuốn sách khác.
- So sánh cuốn sách với từng series bên trong list. Nếu độ tương thích dưới 80% sẽ add tên đó vào series. Ngược lại ta sẽ thêm vào thì ta không thêm vào.
- Với mỗi series trong list sẽ được đánh dấu 1 index, ta gọi hàm index của list để lấy index đó và gán cho dòng của cuốn sách đó.<br>

<font color = red>Hàm trả về</font>: Hàm trả về list là index của series mà cuốn sách thuộc về

In [38]:
# ulen_names = lightnovel_df.name.nunique() # number of unique names
# u_names = list(map(lambda x: x.upper(), lightnovel_df.name.unique())) #array unique names


# ln_series = []
# series_Series = []
# ln_series.append(u_names[0])
# for name in u_names:
#     ratio_series = []
#     for series in ln_series:
#         ratio_series.append(SequenceMatcher(None, series, name).ratio())
#     max_ratio = max(ratio_series)
#     if max_ratio < 0.6:
#         ln_series.append(name)
#         series_Series.append(ln_series.index(name))
#     else:
#         series_Series.append(ratio_series.index(max_ratio))
def categorical_book_name(book_df):
    u_names = list(map(lambda x: x.upper(), list(book_df.name)))
    book_series = []
    series = []
    book_series.append(u_names[0])
    for name in u_names:
        ratio_series = process.extract(name, book_series, scorer = fuzz.token_sort_ratio)
        best_ratio_series = ratio_series[0]
        if best_ratio_series[-1] < 65: # Xét tỉ lệ hợp lệ cao hơn 80%
            book_series.append(name)
            series.append(book_series.index(name))
        else:
            series.append(book_series.index(best_ratio_series[0]))
    return series

Tên các loại sách

In [39]:
list(df.categories.unique())

['Du ký',
 'Light novel',
 'Phê Bình - Lý Luận Văn Học',
 'Phóng sự - Ký sự - Bút ký',
 'Tác phẩm kinh điển',
 'Thơ',
 'Tiểu sử - Hồi ký',
 'Tiểu Thuyết',
 'Truyện Giả tưởng - Huyền Bí - Phiêu Lưu',
 'Tranh Truyện',
 'Truyện cổ tích - Ngụ ngôn',
 'Truyện cười',
 'Truyện dài',
 'Bài học kinh doanh',
 'Truyện đam mỹ',
 'Truyện kiếm hiệp',
 'Truyện kinh dị',
 'Truyện ngắn - Tản văn - Tạp Văn',
 'Truyện ngôn tình',
 'Truyện trinh thám',
 'Ca dao - Tục ngữ',
 'Sách ảnh',
 'Sách doanh nhân',
 'Sách khởi nghiệp',
 'Sách kinh tế học',
 'Sách kỹ năng làm việc',
 'Sách Marketing - Bán hàng ',
 'Sách quản trị, lãnh đạo',
 'Sách quản trị nhân lực',
 'Sách tài chính, kế toán',
 'Sách tài chính, tiền tệ',
 'Đạo đức - Kỹ năng sống',
 'Kiến thức - Bách khoa',
 'Tô màu - Luyện chữ  ',
 'Truyện cổ tích',
 'Truyện kể cho bé',
 'Truyện tranh Ehon',
 'Truyện tranh thiếu nhi',
 'Văn học thiếu nhi',
 'Sách tư duy - Kỹ năng sống',
 'Sách nghệ thuật sống đẹp',
 'Sách hướng nghiệp - Kỹ năng mềm',
 'Sách Thai Gi

Thử với loại sách là `Light Novel`

**Bước 1** : Chọn loại sách `Light Novel` để thực hiện thống kê 

In [40]:
lightnovel_df = get_df_by_categories("Light novel")
# list(lightnovel_df.name)

**Bước 2**: Tiến hành phân loại theo tên sách

In [41]:
series = categorical_book_name(lightnovel_df)

In [42]:
# series = pd.DataFrame({"Series": list(categoricate_book_name(lightnovel_df))})

**Bước 3**: Thêm cột `Series` sau khi phân loại 

In [43]:
lightnovel_df.insert(len(lightnovel_df.columns), "Series", series)
# lightnovel_df = pd.concat([lightnovel_df, series], axis = 1 ,ignore_index=True)

In [44]:
lightnovel_df[lightnovel_df.Series == 0]

,id,sku,name,short_url,book_cover,price,original_price,discount_rate,rating_average,review_count,...,authors,publisher_vn,publication_date,dimensions,manufacturer,number_of_page,dich_gia,edition,luu_y,Series
102,203957932,4966276852148,Chào Mừng Đến Lớp Học Đề Cao Thực Lực - 4,https://tiki.vn/product-p203957932.html?spid=2...,Bìa mềm,95900,120000,20,5.0,25,...,Kinugasa Syougo,IPM,2022-11-11 00:00:00,13x18 cm,Nhà Xuất Bản Hà Nội,356,Thu Hằng,NaN,NaN,0
121,195969669,9063209978122,Chào Mừng Đến Lớp Học Đề Cao Thực Lực - 3,https://tiki.vn/product-p195969669.html?spid=1...,Bìa mềm,95900,120000,20,4.8,142,...,Kinugasa Syougo,IPM,2022-08-08 00:00:00,18x23 cm,Nhà Xuất Bản Hà Nội,380,Hoàng Ngọc,NaN,NaN,0


Thử loại sách `Du ký`

In [45]:
duky_df = get_df_by_categories("Du ký")

In [46]:
series = categorical_book_name(duky_df)
series

[0,
 0,
 1,
 0,
 0,
 2,
 3,
 4,
 5,
 6,
 0,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 14,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 15,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 15,
 41,
 15,
 42,
 43,
 37,
 44,
 45,
 46,
 25,
 47,
 48,
 4,
 46,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 37,
 58,
 59,
 60,
 61,
 62,
 60,
 63,
 46,
 64,
 62,
 57,
 65,
 66,
 23,
 67,
 68,
 69,
 70,
 23,
 71,
 72,
 37,
 37,
 37,
 37,
 73,
 74,
 75,
 76,
 77,
 74]

In [47]:
duky_df.insert(len(duky_df.columns), "Series", series)

In [48]:
duky_df[duky_df.Series == 0]
# duky_df[duky_df.authors == "Trần Đặng Đăng Khoa"]

,id,sku,name,short_url,book_cover,price,original_price,discount_rate,rating_average,review_count,...,authors,publisher_vn,publication_date,dimensions,manufacturer,number_of_page,dich_gia,edition,luu_y,Series
0,204317934,9972713357587,1111 - Nhật Ký Sáu Vạn Dặm Trên Yên Xe Cà Tàng,https://tiki.vn/product-p204317934.html?spid=2...,NaN,262000,325000,19,5.0,55,...,Trần Đặng Đăng Khoa,NXB Trẻ,2022-11-10 00:00:00,13 x 20,NXB Trẻ,0,NaN,NaN,NaN,0
1,205279963,7618382614299,1111 - Nhật Ký Sáu Vạn Dặm Trên Yên Xe Cà Tàng,https://tiki.vn/product-p205279963.html?spid=2...,Bìa mềm,256750,325000,21,0.0,0,...,Trần Đặng Đăng Khoa,NXB Trẻ,2022-11-11 17:44:46,NaN,NXB Trẻ,778,NaN,NaN,NaN,0
3,205288789,9035774965791,1111 - Nhật Ký Sáu Vạn Dặm Trên Yên Xe Cà Tàng,https://tiki.vn/product-p205288789.html?spid=2...,Bìa mềm,292500,325000,10,0.0,0,...,Trần Đặng Đăng Khoa,Văn Lang,2022-11-10 20:17:06,NaN,Nhà Xuất Bản Hồng Đức,776,NaN,NaN,NaN,0
4,204649516,8708257676679,Sách - 1111 - NHẬT KÝ SÁU VẠN DẶM TRÊN YÊN XE ...,https://tiki.vn/product-p204649516.html?spid=2...,NaN,292500,325000,10,0.0,0,...,NaN,NXB Trẻ,NaN,NaN,NXB Trẻ,0,NaN,NaN,NaN,0
10,205293454,4751431039506,1111 - Nhật Ký Sáu Vạn Dặm Trên Yên Xe Cà Tàng,https://tiki.vn/product-p205293454.html?spid=2...,Bìa mềm,256750,325000,21,0.0,0,...,Trần Đặng Đăng Khoa,NXB Trẻ,2022-11-11 00:34:17,NaN,NXB Trẻ,778,NaN,NaN,NaN,0


Thử loại sách `Du ký`

In [49]:
duky_df = get_df_by_categories("Du ký")

In [50]:
series = categorical_book_name(duky_df)
series

[0,
 0,
 1,
 0,
 0,
 2,
 3,
 4,
 5,
 6,
 0,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 14,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 15,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 15,
 41,
 15,
 42,
 43,
 37,
 44,
 45,
 46,
 25,
 47,
 48,
 4,
 46,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 37,
 58,
 59,
 60,
 61,
 62,
 60,
 63,
 46,
 64,
 62,
 57,
 65,
 66,
 23,
 67,
 68,
 69,
 70,
 23,
 71,
 72,
 37,
 37,
 37,
 37,
 73,
 74,
 75,
 76,
 77,
 74]

In [51]:
duky_df.insert(len(duky_df.columns), "Series", series)

In [52]:
duky_df[duky_df.Series == 0]
# duky_df[duky_df.authors == "Trần Đặng Đăng Khoa"]

,id,sku,name,short_url,book_cover,price,original_price,discount_rate,rating_average,review_count,...,authors,publisher_vn,publication_date,dimensions,manufacturer,number_of_page,dich_gia,edition,luu_y,Series
0,204317934,9972713357587,1111 - Nhật Ký Sáu Vạn Dặm Trên Yên Xe Cà Tàng,https://tiki.vn/product-p204317934.html?spid=2...,NaN,262000,325000,19,5.0,55,...,Trần Đặng Đăng Khoa,NXB Trẻ,2022-11-10 00:00:00,13 x 20,NXB Trẻ,0,NaN,NaN,NaN,0
1,205279963,7618382614299,1111 - Nhật Ký Sáu Vạn Dặm Trên Yên Xe Cà Tàng,https://tiki.vn/product-p205279963.html?spid=2...,Bìa mềm,256750,325000,21,0.0,0,...,Trần Đặng Đăng Khoa,NXB Trẻ,2022-11-11 17:44:46,NaN,NXB Trẻ,778,NaN,NaN,NaN,0
3,205288789,9035774965791,1111 - Nhật Ký Sáu Vạn Dặm Trên Yên Xe Cà Tàng,https://tiki.vn/product-p205288789.html?spid=2...,Bìa mềm,292500,325000,10,0.0,0,...,Trần Đặng Đăng Khoa,Văn Lang,2022-11-10 20:17:06,NaN,Nhà Xuất Bản Hồng Đức,776,NaN,NaN,NaN,0
4,204649516,8708257676679,Sách - 1111 - NHẬT KÝ SÁU VẠN DẶM TRÊN YÊN XE ...,https://tiki.vn/product-p204649516.html?spid=2...,NaN,292500,325000,10,0.0,0,...,NaN,NXB Trẻ,NaN,NaN,NXB Trẻ,0,NaN,NaN,NaN,0
10,205293454,4751431039506,1111 - Nhật Ký Sáu Vạn Dặm Trên Yên Xe Cà Tàng,https://tiki.vn/product-p205293454.html?spid=2...,Bìa mềm,256750,325000,21,0.0,0,...,Trần Đặng Đăng Khoa,NXB Trẻ,2022-11-11 00:34:17,NaN,NXB Trẻ,778,NaN,NaN,NaN,0
